# Company Law Retrieval Augmented Generation (RAG) System

This notebook implements a comprehensive RAG system for Company Law 2013 , allowing users to:
- Upload and index PDF documents containing business law content
- Ask questions about business law concepts using natural language
- Get precise answers from document content or relevant web searches

The system uses state-of-the-art LLMs, vector search, and a graph-based pipeline for effective information retrieval.

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
import subprocess

def run(commands):
    for command in commands:
        with subprocess.Popen(command, shell = True, stdout = subprocess.PIPE, stderr = subprocess.STDOUT, bufsize = 1) as sp:
            for line in sp.stdout:
                line = line.decode("utf-8", errors = "replace")
                if "undefined reference" in line:
                    raise RuntimeError("Failed Processing.")
                print(line, flush = True, end = "")
        pass
    pass
pass

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/opoptop/Book.pdf


## Environment Setup

Let's start by installing the necessary libraries for our RAG system. We'll need:
- LangChain and related packages for orchestrating the RAG workflow
- Vector storage with FAISS for efficient retrieval
- PDF processing libraries
- LLM providers (Groq, Google Gemini) for inference
- Gradio for building the web interface

In [2]:
pip install langchain langgraph langchain-experimental langchain-core langchain-community langchain-groq langchain-google-genai langchain-ollama faiss-cpu pypdf gradio python-dotenv tavily-python tiktoken uuid

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.6/91.6 kB 3.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 145.2/145.2 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.2/209.2 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 47.9 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 46.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 433.9/433.9 kB 26.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 61.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.9/46.9 MB 38.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.2/322.2 kB 18.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.

In [3]:
import uuid
import logging
import base64
logger = logging.getLogger(__name__)
from dotenv import load_dotenv
from typing import Annotated, Dict, Any, Optional, List, Sequence, Literal, TypedDict
from typing_extensions import TypedDict
from pydantic import BaseModel as PydanticBaseModel, Field # Alias Pydantic BaseModel

# --- LangChain Core Imports ---
from langchain_core.messages import BaseMessage, HumanMessage, AIMessage
from langchain_core.prompts import ChatPromptTemplate, PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain.schema import Document

# --- Document Loaders and Splitters ---
from langchain_community.document_loaders import PyPDFLoader
from langchain_experimental.text_splitter import SemanticChunker

# --- Vector Stores and Embeddings ---
from langchain_community.vectorstores import FAISS
# Ensure you have Ollama running and the model pulled
# ollama pull nomic-embed-text
from langchain_ollama import OllamaEmbeddings # Correct import

# --- LLMs ---
# Ensure you have GROQ and Google API keys set in your environment
from langchain_groq import ChatGroq
from langchain_google_genai import ChatGoogleGenerativeAI

# --- Tools ---
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain.tools.retriever import create_retriever_tool

# --- LangGraph ---
from langgraph.graph import StateGraph, END, START
from langgraph.graph.message import add_messages
from langgraph.checkpoint.memory import MemorySaver # In-memory checkpointer

# --- Gradio ---
import gradio as gr

## Setting Up Embeddings and LLM Models

In this section, we'll set up the Ollama embedding model that will help us create vector representations of our business law documents. We'll install Ollama and pull the `nomic-embed-text` model for high-quality text embeddings.


In [4]:
commands = [
        "curl -fsSL https://ollama.com/install.sh | sh",
]
run(commands)

import os
os.system("/usr/local/bin/ollama serve &")
os.system("echo 'ollama test'")

>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle


/usr/lib/python3.11/subprocess.py:1016: RuntimeWarning: line buffering (buffering=1) isn't supported in binary mode, the default buffer size will be used
  self.stdout = io.open(c2pread, 'rb', bufsize)


############################################################################################# 100.0%
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.
ollama test


0

In [5]:
commands = [
        "ollama pull nomic-embed-text"
]
run(commands)

Couldn't find '/root/.ollama/id_ed25519'. Generating new private key.
Your new public key is: 

ssh-ed25519 AAAAC3NzaC1lZDI1NTE5AAAAIILSPLRz7226ix7oKGZls3677qtq+uMOx0S+R7glV9Ao

[GIN] 2025/04/19 - 09:55:56 | 200 |      50.997µs |       127.0.0.1 | HEAD     "/"


2025/04/19 09:55:56 routes.go:1231: INFO server config env="map[CUDA_VISIBLE_DEVICES: GPU_DEVICE_ORDINAL: HIP_VISIBLE_DEVICES: HSA_OVERRIDE_GFX_VERSION: HTTPS_PROXY: HTTP_PROXY: NO_PROXY: OLLAMA_CONTEXT_LENGTH:2048 OLLAMA_DEBUG:false OLLAMA_FLASH_ATTENTION:false OLLAMA_GPU_OVERHEAD:0 OLLAMA_HOST:http://127.0.0.1:11434 OLLAMA_INTEL_GPU:false OLLAMA_KEEP_ALIVE:5m0s OLLAMA_KV_CACHE_TYPE: OLLAMA_LLM_LIBRARY: OLLAMA_LOAD_TIMEOUT:5m0s OLLAMA_MAX_LOADED_MODELS:0 OLLAMA_MAX_QUEUE:512 OLLAMA_MODELS:/root/.ollama/models OLLAMA_MULTIUSER_CACHE:false OLLAMA_NEW_ENGINE:false OLLAMA_NOHISTORY:false OLLAMA_NOPRUNE:false OLLAMA_NUM_PARALLEL:0 OLLAMA_ORIGINS:[http://localhost https://localhost http://localhost:* https://localhost:* http://127.0.0.1 https://127.0.0.1 http://127.0.0.1:* https://127.0.0.1:* http://0.0.0.0 https://0.0.0.0 http://0.0.0.0:* https://0.0.0.0:* app://* file://* tauri://* vscode-webview://* vscode-file://*] OLLAMA_SCHED_SPREAD:false ROCR_VISIBLE_DEVICES: http_proxy: https_proxy:

pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest 
pulling 970aa74c0a90...   0% ▕                ▏    0 B/274 MB                  pulling manifest 


time=2025-04-19T09:55:57.664Z level=INFO source=download.go:177 msg="downloading 970aa74c0a90 in 3 100 MB part(s)"


pulling 970aa74c0a90...   0% ▕                ▏    0 B/274 MB                  pulling manifest 
pulling 970aa74c0a90...   4% ▕                ▏  11 MB/274 MB                  pulling manifest 
pulling 970aa74c0a90...  18% ▕██              ▏  50 MB/274 MB                  pulling manifest 
pulling 970aa74c0a90...  24% ▕███             ▏  65 MB/274 MB                  pulling manifest 
pulling 970aa74c0a90...  39% ▕██████          ▏ 106 MB/274 MB                  pulling manifest 
pulling 970aa74c0a90...  56% ▕████████        ▏ 153 MB/274 MB                  pulling manifest 
pulling 970aa74c0a90...  64% ▕██████████      ▏ 175 MB/274 MB                  pulling manifest 
pulling 970aa74c0a90...  74% ▕███████████     ▏ 201 MB/274 MB                  pulling manifest 
pulling 970aa74c0a90...  82% ▕█████████████   ▏ 226 MB/274 MB                  pulling manifest 
pulling 970aa74c0a90...  85% ▕█████████████   ▏ 233 MB/274 MB                  pulling manifest 
pulling 970aa74c0a90...  92% ▕

...

pulling c71d239df917... 100% ▕████████████████▏  11 KB                         pulling manifest 
pulling 970aa74c0a90... 100% ▕████████████████▏ 274 MB                         
pulling c71d239df917... 100% ▕████████████████▏  11 KB                         pulling manifest 
pulling 970aa74c0a90... 100% ▕████████████████▏ 274 MB                         
pulling c71d239df917... 100% ▕████████████████▏  11 KB                         pulling manifest 
pulling 970aa74c0a90... 100% ▕████████████████▏ 274 MB                         
pulling c71d239df917... 100% ▕████████████████▏  11 KB                         pulling manifest 
pulling 970aa74c0a90... 100% ▕████████████████▏ 274 MB                         
pulling c71d239df917... 100% ▕████████████████▏  11 KB                         pulling manifest 
pulling 970aa74c0a90... 100% ▕████████████████▏ 274 MB                         
pulling c71d239df917... 100% ▕████████████████▏  11 KB                         pulling manifest 
pulling 970aa74c0a

time=2025-04-19T09:56:00.952Z level=INFO source=download.go:177 msg="downloading ce4a164fc046 in 1 17 B part(s)"


pulling ce4a164fc046... 100% ▕████████████████▏   17 B                         pulling manifest 
pulling 970aa74c0a90... 100% ▕████████████████▏ 274 MB                         
pulling c71d239df917... 100% ▕████████████████▏  11 KB                         
pulling ce4a164fc046... 100% ▕████████████████▏   17 B                         pulling manifest 
pulling 970aa74c0a90... 100% ▕████████████████▏ 274 MB                         
pulling c71d239df917... 100% ▕████████████████▏  11 KB                         
pulling ce4a164fc046... 100% ▕████████████████▏   17 B                         pulling manifest 
pulling 970aa74c0a90... 100% ▕████████████████▏ 274 MB                         
pulling c71d239df917... 100% ▕████████████████▏  11 KB                         
pulling ce4a164fc046... 100% ▕████████████████▏   17 B                         pulling manifest 
pulling 970aa74c0a90... 100% ▕████████████████▏ 274 MB                         
pulling c71d239df917... 100% ▕████████████████▏  11 

time=2025-04-19T09:56:02.091Z level=INFO source=download.go:177 msg="downloading 31df23ea7daa in 1 420 B part(s)"


pulling 31df23ea7daa... 100% ▕████████████████▏  420 B                         pulling manifest 
pulling 970aa74c0a90... 100% ▕████████████████▏ 274 MB                         
pulling c71d239df917... 100% ▕████████████████▏  11 KB                         
pulling ce4a164fc046... 100% ▕████████████████▏   17 B                         
pulling 31df23ea7daa... 100% ▕████████████████▏  420 B                         pulling manifest 
pulling 970aa74c0a90... 100% ▕████████████████▏ 274 MB                         
pulling c71d239df917... 100% ▕████████████████▏  11 KB                         
pulling ce4a164fc046... 100% ▕████████████████▏   17 B                         
pulling 31df23ea7daa... 100% ▕████████████████▏  420 B                         pulling manifest 
pulling 970aa74c0a90... 100% ▕████████████████▏ 274 MB                         
pulling c71d239df917... 100% ▕████████████████▏  11 KB                         
pulling ce4a164fc046... 100% ▕████████████████▏   17 B               

## Document Processing and Vector Store Setup

In this section, we'll implement the document processing pipeline for business law PDFs. The process includes:
1. Loading documents using PyPDFLoader
2. Chunking the text into semantically meaningful segments
3. Creating vector embeddings using our Ollama model
4. Storing the embeddings in a FAISS vector database for efficient retrieval

In [6]:
embed=OllamaEmbeddings(model='nomic-embed-text')
pdf=PyPDFLoader("/kaggle/input/opoptop/Book.pdf")
chunks=SemanticChunker(embeddings=embed)
pdf_documents = pdf.load()
# Then split the loaded documents using the previously defined chunker
docs = chunks.split_documents(pdf_documents)

time=2025-04-19T09:56:19.262Z level=WARN source=types.go:524 msg="invalid option provided" option=tfs_z
time=2025-04-19T09:56:19.378Z level=WARN source=ggml.go:152 msg="key not found" key=nomic-bert.vision.block_count default=0
time=2025-04-19T09:56:19.378Z level=WARN source=ggml.go:152 msg="key not found" key=nomic-bert.attention.head_count_kv default=1
time=2025-04-19T09:56:19.378Z level=WARN source=ggml.go:152 msg="key not found" key=nomic-bert.attention.key_length default=64
time=2025-04-19T09:56:19.378Z level=WARN source=ggml.go:152 msg="key not found" key=nomic-bert.attention.value_length default=64
time=2025-04-19T09:56:19.379Z level=WARN source=ggml.go:152 msg="key not found" key=nomic-bert.attention.head_count_kv default=1
time=2025-04-19T09:56:19.379Z level=INFO source=sched.go:716 msg="new model will fit in available VRAM in single GPU, loading" model=/root/.ollama/models/blobs/sha256-970aa74c0a90ef7482477cf803618e776e173c007bf957f635f1015bfcfef0e6 gpu=GPU-9709192e-1d5c-b596

[GIN] 2025/04/19 - 09:56:20 | 200 |  1.030483285s |       127.0.0.1 | POST     "/api/embed"


time=2025-04-19T09:56:20.309Z level=WARN source=types.go:524 msg="invalid option provided" option=tfs_z


[GIN] 2025/04/19 - 09:56:20 | 200 |  263.205262ms |       127.0.0.1 | POST     "/api/embed"
[GIN] 2025/04/19 - 09:56:20 | 200 |  154.633746ms |       127.0.0.1 | POST     "/api/embed"


time=2025-04-19T09:56:20.584Z level=WARN source=types.go:524 msg="invalid option provided" option=tfs_z
time=2025-04-19T09:56:20.742Z level=WARN source=types.go:524 msg="invalid option provided" option=tfs_z


...

time=2025-04-19T10:02:46.398Z level=WARN source=types.go:524 msg="invalid option provided" option=tfs_z
time=2025-04-19T10:02:46.486Z level=WARN source=types.go:524 msg="invalid option provided" option=tfs_z
time=2025-04-19T10:02:46.603Z level=WARN source=types.go:524 msg="invalid option provided" option=tfs_z
time=2025-04-19T10:02:46.667Z level=WARN source=types.go:524 msg="invalid option provided" option=tfs_z
time=2025-04-19T10:02:46.733Z level=WARN source=types.go:524 msg="invalid option provided" option=tfs_z


[GIN] 2025/04/19 - 10:02:46 | 200 |   61.159822ms |       127.0.0.1 | POST     "/api/embed"
[GIN] 2025/04/19 - 10:02:46 | 200 |    65.93846ms |       127.0.0.1 | POST     "/api/embed"
[GIN] 2025/04/19 - 10:02:46 | 200 |   88.830218ms |       127.0.0.1 | POST     "/api/embed"


time=2025-04-19T10:02:46.827Z level=WARN source=types.go:524 msg="invalid option provided" option=tfs_z
time=2025-04-19T10:02:46.887Z level=WARN source=types.go:524 msg="invalid option provided" option=tfs_z
time=2025-04-19T10:02:46.984Z level=WARN source=types.go:524 msg="invalid option provided" option=tfs_z


[GIN] 2025/04/19 - 10:02:46 | 200 |   59.004501ms |       127.0.0.1 | POST     "/api/embed"
[GIN] 2025/04/19 - 10:02:46 | 200 |   94.964035ms |       127.0.0.1 | POST     "/api/embed"
[GIN] 2025/04/19 - 10:02:47 | 200 |  109.765569ms |       127.0.0.1 | POST     "/api/embed"
[GIN] 2025/04/19 - 10:02:47 | 200 |   112.77491ms |       127.0.0.1 | POST     "/api/embed"


time=2025-04-19T10:02:47.098Z level=WARN source=types.go:524 msg="invalid option provided" option=tfs_z


## Creating and Saving the Vector Store

With our documents properly chunked, we now create a FAISS vector database from these documents. FAISS is a library for efficient similarity search that will allow us to quickly retrieve the most relevant document chunks based on user queries.

We'll save the vector store locally so it can be reused for future queries without reprocessing the document.

In [7]:
vectorstore = FAISS.from_documents(docs, embed)
vectorstore.save_local("cadb")
vectorstore.as_retriever()

time=2025-04-19T10:02:47.306Z level=WARN source=types.go:524 msg="invalid option provided" option=tfs_z


[GIN] 2025/04/19 - 10:04:08 | 200 |         1m21s |       127.0.0.1 | POST     "/api/embed"


VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x7e0a6db8f990>, search_kwargs={})

## API Configuration and Environment Setup

Next, we'll set up the API keys and environment variables needed for our various services:
- Google API for Gemini LLM
- Groq API for fast LLM inference
- LangChain API for tracking
- Tavily API for web search capabilities

We also define important global variables and configuration for the RAG system.

In [16]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
GOOGLE_API_KEY = user_secrets.get_secret("GOOGLE_API_KEY")
GROQ_API_KEY = user_secrets.get_secret("GROQ_API_KEY")
TAVILY_API_KEY = user_secrets.get_secret("TAVILY_API_KEY")

# --- Global Variables & Configuration ---
VECTOR_STORE_PATH = "cadb"
OLLAMA_EMBED_MODEL = "nomic-embed-text" # Ollama model for embeddings
GROQ_MODEL = 'meta-llama/llama-4-scout-17b-16e-instruct' # Groq model
GEMINI_MODEL = "gemini-2.0-flash" # Google model


# Increased temperatures for more detailed and creative responses
llm_fast = ChatGroq(model=GROQ_MODEL, temperature=0.4, api_key=GROQ_API_KEY) 
llm_gen = ChatGoogleGenerativeAI(model=GEMINI_MODEL, temperature=0.5, api_key=GOOGLE_API_KEY) 



# Increased max_results to get more comprehensive web search results
search_tool = TavilySearchResults(max_results=4, search_depth='advanced', tavily_api_key=TAVILY_API_KEY)

# Initialize global variables
global_retriever = None
indexing_done = False

# Try to load existing vector store if available
try:
    vectorstore = FAISS.load_local(VECTOR_STORE_PATH, OllamaEmbeddings(model=OLLAMA_EMBED_MODEL))
    global_retriever = vectorstore.as_retriever(search_kwargs={"k": 5}) # Retrieve more documents
    indexing_done = True
    logger.info(f"Loaded existing vector store from {VECTOR_STORE_PATH}")
except Exception as e:
    logger.warning(f"Could not load existing vector store: {e}")
    global_retriever = None
    indexing_done = False

## LangGraph RAG Workflow Implementation

Now we'll build the core RAG workflow using LangGraph, a library for building stateful, multi-step workflows for LLM applications. Our workflow consists of these nodes:

1. **Query Rewriting** - Makes user questions more precise for legal document retrieval
2. **Document Retrieval** - Fetches relevant chunks from our vector store
3. **Document Grading** - Evaluates if retrieved documents are relevant
4. **Web Search Fallback** - Searches the web when our documents aren't sufficient
5. **Answer Generation** - Creates comprehensive responses from retrieved context

Each step can be monitored, debugged, and optimized independently, making our system more robust.

In [17]:
class RAGGraphState(TypedDict):
    """State for the RAG pipeline."""
    original_question: str
    question: str
    documents: List[Document]
    is_rag_relevant: bool | None
    generation: str
    source_used: Literal["Vector Store", "Web Search", "None"]

class GradeDocuments(PydanticBaseModel):
    """Binary score for relevance check."""
    binary_score: Literal['yes', 'no'] = Field(description="Is the document relevant? ('yes' or 'no')")

def rewrite_query_node(state: RAGGraphState) -> RAGGraphState:
    """Rewrites the query for legal context retrieval."""
    logger.info("--- RAG Node: REWRITE QUERY ---")
    original_question = state['original_question']
    rewrite_prompt = ChatPromptTemplate.from_messages([
        ("system", "You are a query rewriting expert specializing in Indian the Companies Act. Rewrite the user's question to be specific and focused for retrieving legal documents. Include key legal terms, section numbers, and company law concepts. Use more keywords and be very specific. DO NOT generalize the query."),
        ("human", "Original question: {question}\n\nRewritten query for legal database:"),
    ])
    rewriter_chain = rewrite_prompt | llm_fast | StrOutputParser()
    try:
        rewritten_question = rewriter_chain.invoke({"question": original_question})
        logger.info(f"Rewritten Query: {rewritten_question}")
        return {"question": rewritten_question, "original_question": original_question}
    except Exception as e:
         logger.error(f"Error rewriting query: {e}", exc_info=True)
         return {"question": original_question, "original_question": original_question} # Fallback


def retrieve_rag_node(state: RAGGraphState) -> RAGGraphState:
    """Retrieves documents using the global retriever."""
    logger.info("--- RAG Node: RETRIEVE (Vector Store) ---")
    if global_retriever is None:
        logger.warning("Retriever not available. Skipping RAG retrieval.")
        return {"documents": [], "source_used": "None"}
    question = state['question']
    try:
        logger.info(f"Retrieving documents for: {question}")
        docs = global_retriever.invoke(question)
        logger.info(f"Retrieved {len(docs)} documents from vector store.")
        return {"documents": docs, "source_used": "Vector Store"}
    except Exception as e:
         logger.error(f"Error during RAG retrieval: {e}", exc_info=True)
         return {"documents": [], "source_used": "None"}

def grade_documents_node(state: RAGGraphState) -> RAGGraphState:
    """Grades the relevance of retrieved documents."""
    logger.info("--- RAG Node: GRADE DOCUMENTS ---")
    question = state['question']
    documents = state['documents']
    if not documents:
        logger.warning("No documents to grade.")
        return {"is_rag_relevant": False}

    system = """You are a grader assessing the relevance of a retrieved document snippet to a user question about Indian Business Law or the Companies Act.
    Be LENIENT in your assessment. Even if the document only provides PARTIAL information or CONTEXT that might be helpful, consider it relevant.
    If it contains ANY keywords, legal concepts, section numbers, or related context, consider it relevant.
    Output 'yes' if there's ANY chance the document could help, even partially. Only output 'no' for completely unrelated content."""
    grade_prompt = ChatPromptTemplate.from_messages([
        ("system", system),
        ("human", "Document snippet:\n\n{document}\n\nUser question: {question}"),
    ])
    structured_llm_grader = llm_fast.with_structured_output(GradeDocuments)
    grader_chain = grade_prompt | structured_llm_grader

    relevant_docs_found = False
    for doc in documents[:3]: 
        try:
            doc_snippet = doc.page_content[:1200] 
            score = grader_chain.invoke({"question": question, "document": doc_snippet})
            if score.binary_score == 'yes':
                logger.info("--- GRADE: Relevant document found (Vector Store) ---")
                relevant_docs_found = True
                break
            else:
                logger.info("--- GRADE: Document deemed NOT relevant (Vector Store) ---")
        except Exception as e:
            logger.error(f"Error grading document: {e}", exc_info=True)
            
            continue

    logger.info(f"Vector Store Relevance Decision: {'Relevant' if relevant_docs_found else 'Not Relevant'}")
    return {"is_rag_relevant": relevant_docs_found}

def web_search_node(state: RAGGraphState) -> RAGGraphState:
    """Performs web search if RAG fails."""
    logger.info("--- RAG Node: WEB SEARCH ---")
    question = state['original_question'] 
    logger.info(f"Performing web search for: {question}")
    docs = []
    try:
        enhanced_query = f"Indian Companies Act 2013 {question}"
        web_results = search_tool.invoke({"query": enhanced_query})
        if web_results and isinstance(web_results, list):
             docs = [Document(page_content=d.get("content", ""), metadata={"source": d.get("url", "web")})
                    for d in web_results if d.get("content")]
             logger.info(f"Found {len(docs)} web results.")
        else:
             logger.warning(f"Web search returned unexpected or empty results: {web_results}")
    except Exception as e:
        logger.error(f"Error during web search: {e}", exc_info=True)
        docs = [Document(page_content="Web search failed due to an error.", metadata={"source": "error"})]

    return {"documents": docs, "source_used": "Web Search"}


def generate_answer_node(state: RAGGraphState) -> RAGGraphState:
    """Generates the final answer using context."""
    logger.info("--- RAG Node: GENERATE ANSWER ---")
    question = state['original_question']
    documents = state['documents']
    source = state.get('source_used', 'Unknown') 

    if not documents:
        logger.warning("No documents available for generation.")
        return {"generation": "I don't have specific information about this in my knowledge base. However, regarding your question about Indian company law, I would suggest checking the Companies Act, 2013 which is the primary legislation governing companies in India. You might want to consult a legal professional or check the official Ministry of Corporate Affairs website for the most accurate information."}

    context = "\n\n".join([f"Source: {doc.metadata.get('source', source)}\nContent: {doc.page_content}" for doc in documents])
    prompt_template = """You are an expert assistant specializing in Indian Business Law, with deep knowledge of the Companies Act, 2013, and related topics.

CONTEXT INFORMATION:
{context}

USER QUESTION: {question}

INSTRUCTIONS:
1. Provide a DETAILED and THOROUGH answer based on the CONTEXT INFORMATION above.
2. If the context comes from the "Vector Store" (the uploaded PDF), give preference to that information. If it comes from "Web Search", mention this appropriately.
3. Explain legal concepts in clear terms, with examples where possible.
4. ALWAYS cite relevant section numbers from the Companies Act when applicable.
5. If the context information is insufficient, try to provide a general explanation based on common knowledge of Indian company law.
6. Use proper formatting with headings, paragraphs, and bullet points to organize your answer.
7. Include practical implications of the legal concepts when relevant.
8. If there are opposing viewpoints or interpretations in law, mention them.
9. NEVER say you don't have information unless absolutely necessary - try to provide at least general guidance.

Answer:
"""
    prompt = PromptTemplate(template=prompt_template, input_variables=["context", "question"])
    rag_chain = prompt | llm_gen | StrOutputParser() # Use the more capable LLM for generation

    try:
        generation = rag_chain.invoke({"context": context, "question": question})
    except Exception as e:
        logger.error(f"Error during generation: {e}", exc_info=True)
        generation = "Sorry, an error occurred while generating the response. Please try rephrasing your question."

    logger.info(f"Generated Answer (Source: {source}): {generation[:150]}...")
    return {"generation": generation}

def decide_search_or_generate(state: RAGGraphState) -> Literal["web_search", "generate_answer"]:
    """Decision node for the RAG pipeline."""
    logger.info("--- RAG Edge: Decide Search or Generate ---")
    if state.get('is_rag_relevant', False):
        logger.info("Decision: Relevant docs found in vector store -> Generate Answer.")
        return "generate_answer"
    else:
        logger.info("Decision: No relevant docs in vector store -> Web Search.")
        return "web_search"

## Conversation Management with LangGraph

To create a stateful conversation experience, we build a separate graph that maintains user message history. This architecture separates the RAG pipeline (document retrieval and answer generation) from conversation management, making the system more maintainable.

The conversation graph:
1. Captures incoming user messages
2. Delegates processing to the RAG pipeline
3. Maintains conversation history using checkpointing
4. Provides consistent responses with appropriate context

In [18]:
logger.info("Building the RAG workflow graph...")
rag_workflow = StateGraph(RAGGraphState)
rag_workflow.add_node("rewrite_query", rewrite_query_node)
rag_workflow.add_node("retrieve_rag", retrieve_rag_node)
rag_workflow.add_node("grade_documents", grade_documents_node)
rag_workflow.add_node("web_search", web_search_node)
rag_workflow.add_node("generate_answer", generate_answer_node)

rag_workflow.set_entry_point("rewrite_query")
rag_workflow.add_edge("rewrite_query", "retrieve_rag")
rag_workflow.add_edge("retrieve_rag", "grade_documents")
rag_workflow.add_conditional_edges(
    "grade_documents",
    decide_search_or_generate,
    {"web_search": "web_search", "generate_answer": "generate_answer"}
)
rag_workflow.add_edge("web_search", "generate_answer")
rag_workflow.add_edge("generate_answer", END)

try:
    rag_app = rag_workflow.compile()
    logger.info("RAG workflow graph compiled successfully.")
except Exception as e:
    logger.error(f"Error compiling RAG graph: {e}", exc_info=True)
    rag_app = None
    raise SystemExit("Failed to compile RAG workflow.")
class ConversationState(TypedDict):
    """State for the conversational agent."""
    messages: Annotated[List[BaseMessage], add_messages]
memory = MemorySaver()

def call_rag_pipeline(state: ConversationState) -> ConversationState:
    """Node that invokes the RAG pipeline for the latest user query."""
    logger.info("--- Conversation Node: CALL RAG PIPELINE ---")
    last_message = state['messages'][-1]
    if not isinstance(last_message, HumanMessage):
        logger.warning("Last message is not a HumanMessage, skipping RAG call.")
        return {"messages": [AIMessage(content="Internal error: Expected user input.")]}

    user_query = last_message.content
    logger.info(f"Invoking RAG pipeline for query: {user_query}")

    if rag_app is None:
        logger.error("RAG App is not compiled. Cannot process query.")
        return {"messages": [AIMessage(content="Error: The document processing pipeline is not ready.")]}
    if not indexing_done and global_retriever is None:
         logger.warning("PDF not indexed. RAG pipeline will rely on web search only.")

    rag_input = {"original_question": user_query}
    try:
        rag_output = rag_app.invoke(rag_input)
        generated_answer = rag_output.get("generation", "Sorry, I couldn't retrieve an answer.")
        logger.info("RAG pipeline finished.")
        return {"messages": [AIMessage(content=generated_answer)]}
    except Exception as e:
        logger.error(f"Error invoking RAG pipeline: {e}", exc_info=True)
        return {"messages": [AIMessage(content="Sorry, there was an error retrieving information.")]}
logger.info("Building the conversational workflow graph...")
conversation_graph = StateGraph(ConversationState)
conversation_graph.add_node("call_rag", call_rag_pipeline)
conversation_graph.set_entry_point("call_rag")
conversation_graph.add_edge("call_rag", END)

try:
   
    agent_executor = conversation_graph.compile(checkpointer=memory)
    logger.info("Conversational agent compiled successfully.")
except Exception as e:
    logger.error(f"Error compiling conversational agent: {e}", exc_info=True)
    agent_executor = None
    raise SystemExit("Failed to compile conversational agent.")


## PDF Processing and Document Management

This section defines the functions for processing uploaded PDF documents. The system can:
1. Accept PDF uploads through the Gradio interface
2. Process the PDF content into text chunks
3. Create vector embeddings for each chunk
4. Update the retrieval system with the new document index

This functionality enables users to upload their own Company law document and query them directly.
## Gradio User Interface

Our final section implements the user interface using Gradio, providing an intuitive way for users to interact with the business law RAG system. The interface includes:

1. A PDF upload area for document indexing
2. A conversational chat interface with message history
3. Status indicators showing system state
4. Controls for clearing conversations and managing the interface

The UI is designed to be straightforward while providing all necessary functionality for document management and question answering.

In [19]:
def handle_pdf_upload(pdf_file):
    """Handles PDF upload, processing, and retriever setup. Resets thread_id for a new session."""
    global global_retriever, indexing_done
    if pdf_file is None:
        return "Status: No PDF uploaded. Using existing index if available, or web search only.", None

    file_path = pdf_file.name
    logger.info(f"Processing uploaded PDF: {file_path}")
    status_update = f"Processing PDF: {os.path.basename(file_path)}..."
    yield status_update, None

    vectorstore = process_pdf(file_path)

    if vectorstore:
        global_retriever = vectorstore.as_retriever()
        indexing_done = True
        status_update = f"Status: Successfully indexed {os.path.basename(file_path)}. Ready to chat."
        logger.info("Retriever updated with new PDF.")
        yield status_update, None, str(uuid.uuid4())
    else:
        status_update = f"Status: Failed to index {os.path.basename(file_path)}. Please check logs. Will rely on web search."
        global_retriever = None
        indexing_done = False
        logger.error("Retriever could not be created from PDF.")
        yield status_update, None, str(uuid.uuid4())
def chat_interface(message, history, thread_id_state):
    """Handles the chat interaction with the LangGraph agent."""
    logger.info(f"Received message for thread_id {thread_id_state}: {message}")

    if agent_executor is None:
         logger.error("Agent executor not available.")
         history.append((message, "Error: Chat agent is not ready."))
         return "", history, thread_id_state

    if not indexing_done and global_retriever is None:
         logger.warning("Chatting without an indexed PDF. Relying on web search.")

    config = {"configurable": {"thread_id": thread_id_state}}

    response_content = ""
    result = None
    try:
        logger.info(f"Streaming response for thread_id {thread_id_state}")

        inputs = {"messages": HumanMessage(content=message)}

        result = agent_executor.invoke(inputs, config)

        if result and "messages" in result and len(result["messages"]) > 0:
            ai_msg_obj = result["messages"][-1]
            if isinstance(ai_msg_obj, AIMessage):
                response_content = ai_msg_obj.content
            else:
                response_content = "Error: Agent returned an unexpected message type."
                logger.error(f"Unexpected message type from agent: {type(ai_msg_obj)}")
        else:
            response_content = "Sorry, I couldn't process that."
            logger.error(f"Agent did not return expected messages. Result: {result}")

        logger.info(f"Agent response received for thread_id {thread_id_state}")

    except Exception as e:
        logger.error(f"Error during agent invocation for thread_id {thread_id_state}: {e}", exc_info=True)
        response_content = f"An error occurred: {e}"

    gradio_history = []
    if result and "messages" in result:
        lc_messages = result["messages"]
        for i in range(0, len(lc_messages), 2):
            user_msg = lc_messages[i].content if isinstance(lc_messages[i], HumanMessage) else None
            ai_msg = lc_messages[i+1].content if (i+1 < len(lc_messages)) and isinstance(lc_messages[i+1], AIMessage) else None
            if user_msg is not None:
                gradio_history.append((user_msg, ai_msg))
    else:
        history.append((message, response_content))
        gradio_history = history

    return "", gradio_history, thread_id_state


logger.info("Building Gradio interface...")
with gr.Blocks(theme=gr.themes.Soft(), title="CA Company Law Chat") as demo:
    gr.Markdown("# Chat with Indian Company Law Assistant")
    gr.Markdown("Upload your Company Law and ask questions.")

    thread_id_state = gr.State(value=str(uuid.uuid4()))

    with gr.Row():
        with gr.Column(scale=1):
            pdf_upload = gr.File(label="Upload Business Law PDF", file_types=[".pdf"])
            index_button = gr.Button("Index PDF")
            upload_status = gr.Textbox("Status: Please upload and index PDF.", label="Indexing Status", interactive=False)

        with gr.Column(scale=4):
            chatbot = gr.Chatbot(label="Chat History", height=600)
            msg_textbox = gr.Textbox(label="Your Question", placeholder="Ask about Contracts Act, Sale of Goods Act, etc.", scale=7)
            submit_button = gr.Button("Send", variant="primary", scale=1)
            clear_button = gr.ClearButton([msg_textbox, chatbot], value="Clear Chat")

    upload_event = index_button.click(
        fn=lambda pdf: next(handle_pdf_upload(pdf)),  
        inputs=[pdf_upload],
        outputs=[upload_status, chatbot, thread_id_state] 
    )

    msg_textbox.submit(
         fn=chat_interface,
         inputs=[msg_textbox, chatbot, thread_id_state],
         outputs=[msg_textbox, chatbot, thread_id_state] 
    )
    submit_button.click(
         fn=chat_interface,
         inputs=[msg_textbox, chatbot, thread_id_state],
         outputs=[msg_textbox, chatbot, thread_id_state]
    )

logger.info("Gradio interface defined.")
if __name__ == "__main__":
    logger.info("Launching Gradio App...")
    demo.launch(share=True)

/tmp/ipykernel_31/4048966486.py:95: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot(label="Chat History", height=600)


* Running on local URL:  http://127.0.0.1:7862
* Running on public URL: https://97e276b21532a8f5d9.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
